## Work
1. 請嘗試寫一個 callback 用來記錄各類別在訓練過程中，對驗證集的 True Positive 與 True Negative

In [2]:
import os
import keras
import warnings
warnings.filterwarnings('ignore')

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
train, test = keras.datasets.cifar10.load_data()

In [4]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [5]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [6]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [7]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [14]:
# 載入 Callbacks
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
import numpy as np

# Record_fp_tp
class Record_tp_tn(Callback):
    """Code Here
    撰寫一個紀錄 ture positive 與 true negative 數量的 callbacks    
    """
    def on_train_begin(self, epoch, logs={}):
        logs = logs or {}
        record_items = ["val_auc", "val_f1sc", "val_fp", "val_fn", "val_tp", "val_tn"]
        for i in record_items:
            if i not in self.params['metrics']:
                self.params['metrics'].append(i)
    def on_epoch_end(self, epoch, logs={}, thres=0.5):
        logs = logs or {}
        y_true = self.validation_data[1].argmax(axis = 1)
        y_pred = self.model.predict(self.validation_data[0])
        print("y_pred:", y_pred)
        y_pred = (y_pred[:, 1] >= thres) * 1
        
        #tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        #logs["val_tn"] = tn 
        #logs["val_fp"] = fp
        logs["valid_tp"] = np.sum(np.logical_and(y_pred == 1, y_true == 1))
        logs["valid_tn"] = np.sum(np.logical_and(y_pred == 0, y_true == 0))     
        
rec_tptn = Record_tp_tn()

In [16]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

"""Code Here
將 callbacks 放入模型訓練流程中
"""
model.fit(x_train, y_train,
         epochs=EPOCHS,
         batch_size=BATCH_SIZE,
         validation_data=(x_test, y_test),
         shuffle=True,
         callbacks=[rec_tptn])

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

"""Code Here
將 tp/tn 從訓練完的模型中取出
"""
valid_tp = model.history.history["valid_tp"]
valid_tn = model.history.history["valid_tn"]

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               786688    
_________________________________________________________________
batch_normalization_10 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalization_11 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 256)               65792     
_________________________________________________________________
batch_normalization_12 (Batc (None, 256)               1024

50000/50000 [==============================] - 2s 46us/step - loss: 1.3177 - acc: 0.5411 - val_loss: 1.5033 - val_acc: 0.4705
y_pred: [[0.04133841 0.2874705  0.20578283 ... 0.01905748 0.0154079  0.02880618]
 [0.03928011 0.34562108 0.00566763 ... 0.00077799 0.13885182 0.46248013]
 [0.1787318  0.10931657 0.00222344 ... 0.01799383 0.5453368  0.12825716]
 ...
 [0.00088525 0.00200836 0.41100377 ... 0.03829754 0.00728058 0.00526867]
 [0.03498613 0.02825926 0.12270456 ... 0.10783578 0.00434218 0.02410471]
 [0.01557349 0.00717778 0.09304183 ... 0.23422071 0.01978512 0.01298635]]
Epoch 12/25
50000/50000 [==============================] - 2s 47us/step - loss: 1.2948 - acc: 0.5489 - val_loss: 1.4905 - val_acc: 0.4735
y_pred: [[5.9552260e-02 1.4360692e-01 1.7451926e-01 ... 2.4783058e-02
  2.9504618e-02 2.7248474e-02]
 [1.3515607e-02 2.3607646e-01 1.0882427e-03 ... 2.2631304e-04
  8.6330011e-02 6.6033250e-01]
 [1.7124708e-01 1.9801669e-01 8.3193980e-04 ... 1.0436553e-02
  4.4200242e-01 1.6174485e-0

50000/50000 [==============================] - 2s 50us/step - loss: 1.0805 - acc: 0.6287 - val_loss: 1.4432 - val_acc: 0.4960
y_pred: [[3.14862728e-02 8.86749625e-02 2.89739162e-01 ... 1.65447704e-02
  1.05746211e-02 6.94152415e-02]
 [2.52160206e-02 1.59125298e-01 7.24122045e-04 ... 1.11173278e-04
  1.63266093e-01 6.49428606e-01]
 [2.01686308e-01 5.95882982e-02 1.54915522e-03 ... 3.70189883e-02
  5.71295202e-01 1.13773651e-01]
 ...
 [1.14971411e-03 1.45241676e-03 4.26007748e-01 ... 3.44288200e-02
  1.35431625e-02 2.76502874e-03]
 [4.08592373e-02 3.19469385e-02 1.15408085e-01 ... 5.38167246e-02
  2.32012756e-03 2.33378317e-02]
 [6.68868143e-03 3.62672261e-03 5.65482900e-02 ... 2.58871168e-01
  2.12126076e-02 6.97715394e-03]]
Epoch 24/25
50000/50000 [==============================] - 2s 47us/step - loss: 1.0639 - acc: 0.6343 - val_loss: 1.4450 - val_acc: 0.4988
y_pred: [[1.88355315e-02 8.07120204e-02 1.86478242e-01 ... 1.07374620e-02
  3.82838445e-03 5.30115589e-02]
 [1.96575876e-02 1.06

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(len(train_loss)), train_loss, label="train loss")
plt.plot(range(len(valid_loss)), valid_loss, label="valid loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(range(len(train_acc)), train_acc, label="train accuracy")
plt.plot(range(len(valid_acc)), valid_acc, label="valid accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()

plt.plot(range(len(valid_tp)), valid_tp, label="valid tp", color="navy")
plt.plot(range(len(valid_tn)), valid_tn, label="valid tn", color="red")
plt.legend()
plt.title("True positives and True Negatives")
plt.show()
